In [15]:
import numpy as np
import pandas as pd
from hrvanalysis import remove_outliers, remove_ectopic_beats, interpolate_nan_values
from hrvanalysis import get_time_domain_features
import math
from hrvanalysis import plot_timeseries
import matplotlib.pyplot as plt
import os
import datetime
from datetime import datetime, timedelta

AttributeError: module 'numpy' has no attribute 'asscalar'

In [21]:
import numpy as np
np.version.version

'1.24.4'

In [ ]:
#Modify User to get different file
dfs = pd.read_csv('./Healthy_sleep/DataPaper/user_1/RR.csv')

In [ ]:
Values = list(dfs['ibi_s'])
Values = [i * 1000 for i in Values]
Values = np.sort(Values)

In [ ]:
q1 = np.percentile(Values, 25)
q3 = np.percentile(Values, 75)
iqr = q3 - q1

In [67]:
RR = list(dfs['ibi_s'])
RR = [i * 1000 for i in RR]
dfs['ibi_s'] = remove_outliers(rr_intervals = RR,low_rri = q1-1.5*iqr,high_rri = q3+1.5*iqr,verbose = False)

In [68]:
dfs['ibi_s'] = interpolate_nan_values(rr_intervals=dfs['ibi_s'],interpolation_method="linear")
dfs['ibi_s'] = remove_ectopic_beats(rr_intervals=dfs['ibi_s'], method="malik")
interpolated_nn_intervals = interpolate_nan_values(rr_intervals= dfs['ibi_s'])

2021 ectopic beat(s) have been deleted with malik rule.


In [70]:
dfs = dfs.dropna()

In [71]:
nns = list(dfs['ibi_s'])

In [73]:
Differencing_nn_intervals = [] 
current = nns[0]
for i in range(1,len(nns)):
    Differencing_nn_intervals.append(np.log(nns[i]) - np.log(current))
    current = nns[i]
Differencing_nn_intervals.append(0)
#Differencing_nn_intervals = [x for x in Differencing_nn_intervals if not math.isnan(x)]
print(Differencing_nn_intervals.count('Nan'))
print(len(Differencing_nn_intervals))

0
89836


In [74]:
dfs['ibi_s'] = Differencing_nn_intervals

/tmp/ipykernel_1314957/1611054420.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfs['ibi_s'] = Differencing_nn_intervals


In [59]:
#If we drop duplicates we lose a lot of info
#dfs = dfs.drop_duplicates(subset = 'time')

In [76]:
dfs.drop("Unnamed: 0", axis=1, inplace=True)

/tmp/ipykernel_1314957/266893487.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfs.drop("Unnamed: 0", axis=1, inplace=True)


In [99]:
#Checcking other data: Actigraph (make specific to User)
Filepath = "./Healthy_sleep/DataPaper/user_1/Actigraph.csv"
df = pd.read_csv(Filepath,index_col=0)

In [101]:
final = dfs.merge(df,on=['time','day'],how='left')

In [102]:
final = final.drop(["Vector Magnitude"],axis = 1)

In [104]:
#Checking other data Activity (make specific to User)
Filepath = "./Healthy_sleep/DataPaper/user_1/Activity.csv"
df = pd.read_csv(Filepath)

In [105]:
def calculateTime(start,end):
    start_time = datetime.strptime(start, "%H:%M")
    end_time = datetime.strptime(end, "%H:%M")
    time_list = []
    current_time = start_time

    while current_time <= end_time:
        time_list.append(current_time.strftime('%H:%M:%S'))
        current_time += timedelta(seconds=1)

    return time_list

In [106]:
#Version that adds values between start and end
testdf = pd.DataFrame(columns = ['time','day','Activity'] )

for i in range(0,len(df)):
    if i > 0:
        if df['Activity'][i-1] == 0: #At this point reasonable to assume sleeping
            Activity = 1
            Day = df['Day'][i]
            timelist = calculateTime(end,df['Start'][i])
            for time in timelist:
                new = [time,Day,Activity]
                new = pd.DataFrame(columns=testdf.columns, data=[new])
                testdf = pd.concat([testdf, new], axis=0)
                
        elif df['Activity'][i] != 0:#Otherwise fill in with zero
            Activity = 0
            Day = df['Day'][i]
            timelist = calculateTime(end,df['Start'][i])
            for time in timelist:
                new = [time,Day,Activity]
                new = pd.DataFrame(columns=testdf.columns, data=[new])
                testdf = pd.concat([testdf, new], axis=0)
        else:
            continue
            
    #Add values between start and end
    Activity = df['Activity'][i]
    Day = df['Day'][i]
    timelist = calculateTime(df['Start'][i],df['End'][i])
    for time in timelist:
        new = [time,Day,Activity]
        new = pd.DataFrame(columns=testdf.columns, data=[new])
        testdf = pd.concat([testdf, new], axis=0)
    
    end = df['End'][i]
    

In [32]:
#Original version leaves gaps
testdf = pd.DataFrame(columns = ['time','day','Activity'] )
for i in range(0,len(df)):
    if df['End'].isnull().iloc[i] or df['Start'].isnull().iloc[i]:
        continue
    Activity = df['Activity'][i]
    Day = df['Day'][i]
    timelist = calculateTime(df['Start'][i],df['End'][i])
    for time in timelist:
        new = [time,Day,Activity]
        new = pd.DataFrame(columns=testdf.columns, data=[new])
        testdf = pd.concat([testdf, new], axis=0)

In [107]:
testdf

,time,day,Activity
0,10:00:00,1,2
0,10:00:01,1,2
0,10:00:02,1,2
0,10:00:03,1,2
0,10:00:04,1,2
...,...,...,...
0,09:34:56,2,3
0,09:34:57,2,3
0,09:34:58,2,3
0,09:34:59,2,3


In [108]:
final = final.merge(testdf,on=['time','day'],how='left')

In [109]:
final

,ibi_s,day,time,Steps,HR,Inclinometer Off,Inclinometer Standing,Inclinometer Sitting,Inclinometer Lying,Vector Magnitude,Activity
0,0.107857,1,10:10:18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
1,0.628775,1,10:10:18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
2,0.215669,1,10:10:20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
3,-0.720308,1,10:10:21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
4,-0.109284,1,10:10:23,1.0,68.0,0.0,1.0,0.0,0.0,13.64,2
...,...,...,...,...,...,...,...,...,...,...,...
90730,-0.004246,2,09:45:24,0.0,117.0,0.0,0.0,1.0,0.0,28.30,NaN
90731,-0.010695,2,09:45:25,1.0,119.0,0.0,1.0,0.0,0.0,43.84,NaN
90732,-0.004310,2,09:45:25,1.0,119.0,0.0,1.0,0.0,0.0,43.84,NaN
90733,-0.013044,2,09:45:25,1.0,119.0,0.0,1.0,0.0,0.0,43.84,NaN


In [110]:
#Checcking other data User
Filepath = "./Healthy_sleep/DataPaper/user_1/user_info.csv"
df = pd.read_csv(Filepath)

In [111]:
df_without_na = final.dropna()

In [112]:
df_without_na['Gender'] = df['Gender'][0]
df_without_na['Weight'] = df['Weight'][0]
df_without_na['Height'] = df['Height'][0]
df_without_na['Age'] = df['Age'][0]

/tmp/ipykernel_1314957/2652220233.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_without_na['Gender'] = df['Gender'][0]
/tmp/ipykernel_1314957/2652220233.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_without_na['Weight'] = df['Weight'][0]
/tmp/ipykernel_1314957/2652220233.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pan

In [113]:
df_without_na

,ibi_s,day,time,Steps,HR,Inclinometer Off,Inclinometer Standing,Inclinometer Sitting,Inclinometer Lying,Vector Magnitude,Activity,Gender,Weight,Height,Age
4,-0.109284,1,10:10:23,1.0,68.0,0.0,1.0,0.0,0.0,13.64,2,M,65,169,29
5,-0.162374,1,10:10:23,1.0,68.0,0.0,1.0,0.0,0.0,13.64,2,M,65,169,29
6,-0.119591,1,10:10:24,0.0,68.0,0.0,0.0,1.0,0.0,23.26,2,M,65,169,29
7,0.049255,1,10:10:24,0.0,68.0,0.0,0.0,1.0,0.0,23.26,2,M,65,169,29
8,0.046943,1,10:10:24,0.0,68.0,0.0,0.0,1.0,0.0,23.26,2,M,65,169,29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89832,0.007194,2,09:34:57,0.0,109.0,0.0,0.0,1.0,0.0,0.00,3,M,65,169,29
89833,-0.036500,2,09:34:58,0.0,109.0,0.0,0.0,1.0,0.0,0.00,3,M,65,169,29
89834,-0.007463,2,09:34:58,0.0,109.0,0.0,0.0,1.0,0.0,0.00,3,M,65,169,29
89835,0.011173,2,09:34:59,0.0,109.0,0.0,0.0,1.0,0.0,0.00,3,M,65,169,29


In [114]:
new_order = ['time', 'day'] + [col for col in df_without_na.columns if col not in ['time', 'day']]
df_without_na = df_without_na[new_order]



In [115]:
df_without_na

,time,day,ibi_s,Steps,HR,Inclinometer Off,Inclinometer Standing,Inclinometer Sitting,Inclinometer Lying,Vector Magnitude,Activity,Gender,Weight,Height,Age
4,10:10:23,1,-0.109284,1.0,68.0,0.0,1.0,0.0,0.0,13.64,2,M,65,169,29
5,10:10:23,1,-0.162374,1.0,68.0,0.0,1.0,0.0,0.0,13.64,2,M,65,169,29
6,10:10:24,1,-0.119591,0.0,68.0,0.0,0.0,1.0,0.0,23.26,2,M,65,169,29
7,10:10:24,1,0.049255,0.0,68.0,0.0,0.0,1.0,0.0,23.26,2,M,65,169,29
8,10:10:24,1,0.046943,0.0,68.0,0.0,0.0,1.0,0.0,23.26,2,M,65,169,29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89832,09:34:57,2,0.007194,0.0,109.0,0.0,0.0,1.0,0.0,0.00,3,M,65,169,29
89833,09:34:58,2,-0.036500,0.0,109.0,0.0,0.0,1.0,0.0,0.00,3,M,65,169,29
89834,09:34:58,2,-0.007463,0.0,109.0,0.0,0.0,1.0,0.0,0.00,3,M,65,169,29
89835,09:34:59,2,0.011173,0.0,109.0,0.0,0.0,1.0,0.0,0.00,3,M,65,169,29


In [116]:
df_without_na.to_csv("Patient1Values.csv")

In [230]:
#Okay break for a sec because I want to make the dataframe 3D
df = []
patients = []
for i in range(1,23):
    if i == 15:
        continue
    Filepath = path + str(i)+str('/')
    df.append(pd.read_csv(os.path.join(Filepath, extension)))
    patients.append(i)

In [232]:
dfs = pd.concat(df, ignore_index=True)

In [233]:
Values = list(dfs['ibi_s'])
Values = [i * 1000 for i in Values]
Values = np.sort(Values)

In [234]:
q1 = np.percentile(Values, 25)
q3 = np.percentile(Values, 75)
iqr = q3 - q1

In [236]:
Patient_df = pd.DataFrame()
for i in range(0,21):
    #print(i)
    Patient_number = i + 1
    if i >= 15:
        Patient_number = i + 1
    RR = list(df[i]['ibi_s'])
    RR = [i * 1000 for i in RR]
    rr_intervals_without_outliers = remove_outliers(rr_intervals = RR,low_rri = q1-1.5*iqr,
                                                high_rri = q3+1.5*iqr,verbose = False)
    interpolated_rr_intervals = interpolate_nan_values(rr_intervals=rr_intervals_without_outliers,
                                                   interpolation_method="linear")
    nn_intervals_list = remove_ectopic_beats(rr_intervals=rr_intervals_without_outliers, method="malik")
    interpolated_nn_intervals = interpolate_nan_values(rr_intervals=nn_intervals_list)
    #interpolated_nn_intervals = [x for x in interpolated_nn_intervals if not math.isnan(x)]
    
    
    #Stationary conversion
    Differencing_nn_intervals = [] 
    current = nn_intervals_list[0]
    for i in range(0,len(nn_intervals_list)):
        if current != nn_intervals_list[i]:
            Differencing_nn_intervals.append(np.log(nn_intervals_list[i]) - np.log(current))
            current = nn_intervals_list[i]
    Differencing_nn_intervals = [x for x in Differencing_nn_intervals if not math.isnan(x)]
    #print(Differencing_nn_intervals.count('Nan'))
    #print(len(Differencing_nn_intervals))
    
    new_data = pd.DataFrame({'Patient' + str(Patient_number): Differencing_nn_intervals})
    Patient_df = pd.concat([Patient_df, new_data], ignore_index=True)

2437 ectopic beat(s) have been deleted with malik rule.
0
85185
2496 ectopic beat(s) have been deleted with malik rule.
0
65176
1922 ectopic beat(s) have been deleted with malik rule.
0
78126
13355 ectopic beat(s) have been deleted with malik rule.
0
35410
1780 ectopic beat(s) have been deleted with malik rule.
0
72822
2139 ectopic beat(s) have been deleted with malik rule.
0
86129
453 ectopic beat(s) have been deleted with malik rule.
0
80606
2068 ectopic beat(s) have been deleted with malik rule.
0
66025
934 ectopic beat(s) have been deleted with malik rule.
0
71270
1773 ectopic beat(s) have been deleted with malik rule.
0
63744
762 ectopic beat(s) have been deleted with malik rule.
0
74178
278 ectopic beat(s) have been deleted with malik rule.
0
78745
251 ectopic beat(s) have been deleted with malik rule.
0
79309
292 ectopic beat(s) have been deleted with malik rule.
0
76245
2798 ectopic beat(s) have been deleted with malik rule.
0
71082
2363 ectopic beat(s) have been deleted with m

In [237]:
Patient_df

,Patient1,Patient2,Patient3,Patient4,Patient5,Patient6,Patient7,Patient8,Patient9,Patient10,...,Patient12,Patient13,Patient14,Patient15,Patient16,Patient17,Patient18,Patient19,Patient20,Patient21
0,0.125538,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.041271,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-0.003704,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.009234,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.027200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1529305,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.001241
1529306,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.040127
1529307,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.066527
1529308,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.017677


In [77]:
Patient_df.to_csv("PatientValues.csv")


In [44]:
patients = np.array(patients)
headers = np.array(['ibi_s','day','time']*3)
df = np.array(df)
dfs = pd.DataFrame(df.T,columns=pd.MultiIndex.from_tuples(zip(patients,headers)))

/tmp/ipykernel_2959145/2391729657.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  df = np.array(df)


ValueError: Shape of passed values is (21, 1), indices imply (21, 9)

In [79]:
#My original code
df = []
for i in range(1,23):
    if i == 15:
        continue
    Filepath = path + str(i)+str('/')
    df.append(pd.read_csv(os.path.join(Filepath, extension)))

In [84]:
testdf = list(Patient_df['Patient1'])
print(testdf)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

